<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Sbu-logo.svg/1200px-Sbu-logo.svg.png" alt="keras" width="150" height="150">

<h1 align=center><font size = 7>NLP Summer School</font></h1>
<h1 align=center><font size = 6>NLP Research Lab</font></h1>
<h1 align=center><font size = 5>Shahid Beheshti University</font></h1>
<h1 align=center><font size = 4> July 2022 </font></h1>

## Original Repo
https://github.com/olegborisovv/NGram_LanguageModel

In [ ]:
'''
N-gram_text_generation:
(بایستی قسمت مربوط به اسموتینگ پیاده سازی شود. (10
راهکار شبهه بیم سرچ برای تولید متن (10)

RNN_text_generation:
استفاده از بافتار از 2 کلمه به 6 کلمه ارتقاء پیدا کند (20)
استفاده از روشهای بارگذاری داده متنی (کلاس دیتاست و دیتالودر در پایتورچ) (امتیازی) (10)
RNN_vs_LSTM_vs_GRU:
هر کدام از مدل ها به چند لایه (استکد) بروزرسانی شوند - دراپ اوت، دنس لیر و ... اضافه بشود در صورت لزوم (15)
دقت به بیش از 92 درصد برسد روی داده تست (15)
seq2seq_translation_tutorial:
یک مدل دیگر از مکانیزم توجه پیاده سازی شود - مکانیزم فعلی بر اساس ضرب داخلی عمل میکند (15)
CNN_Text_Classification:
این معماری با یک مدل سکوئنشیال (GRU, LSTM, RNN) ترکیب شود. (15)
مهلت ارسال: ۱۲ شب ۳۱ تیر ماه'''

'\nN-gram_text_generation:\n(بایستی قسمت مربوط به اسموتینگ پیاده سازی شود. (10\nراهکار شبهه بیم سرچ برای تولید متن (10)\n\nRNN_text_generation:\nاستفاده از بافتار از 2 کلمه به 6 کلمه ارتقاء پیدا کند (20)\nاستفاده از روشهای بارگذاری داده متنی (کلاس دیتاست و دیتالودر در پایتورچ) (امتیازی) (10)\nRNN_vs_LSTM_vs_GRU:\nهر کدام از مدل ها به چند لایه (استکد) بروزرسانی شوند - دراپ اوت، دنس لیر و ... اضافه بشود در صورت لزوم (15)\nدقت به بیش از 92 درصد برسد روی داده تست (15)\nseq2seq_translation_tutorial:\nیک مدل دیگر از مکانیزم توجه پیاده سازی شود - مکانیزم فعلی بر اساس ضرب داخلی عمل میکند (15)\nCNN_Text_Classification:\nاین معماری با یک مدل سکوئنشیال (GRU, LSTM, RNN) ترکیب شود. (15)\nمهلت ارسال: ۱۲ شب ۳۱ تیر ماه'

In [ ]:
import string
import random
import time
from typing import List

In [ ]:
# ideally we would use some smart text tokenizer, but for simplicity use this one
def tokenize(text: str) -> List[str]:
    """
    :param text: Takes input sentence
    :return: tokenized sentence
    """
    for punct in string.punctuation:
        text = text.replace(punct, ' '+punct+' ')
    t = text.split()
    
    return t

In [ ]:
def get_ngrams(n: int, tokens: list) -> list:
    """
    :param n: n-gram size
    :param tokens: tokenized sentence
    :return: list of ngrams

    ngrams of tuple form: ((previous wordS!), target word)
    """
    # tokens.append('<END>')
    tokens = (n-1)*['<START>']+tokens
    l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
    return l

In [ ]:
class NgramModel(object):

  def __init__(self, n):
        self.n = n

        # dictionary that keeps list of candidate words given context
        self.context = {}

        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = {}

  def update(self, sentence: str) -> None:
        """
        Updates Language Model
        :param sentence: input text
        """
        n = self.n
        ngrams = get_ngrams(n, tokenize(sentence))
        for ngram in ngrams:
            if ngram in self.ngram_counter:
                self.ngram_counter[ngram] += 1.0
            else:
                self.ngram_counter[ngram] = 1.0

            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]

    # remove try-catch
  '''
  def prob(self, context, token):
        Calculates probability of a candidate token to be generated given a context
        :return: conditional probability
      try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context
        except KeyError:
            result = 0.0
        return result
  '''
     # smooth function prevents the numerator to become zero
  def prob(self, context, token):
        count_of_token = self.ngram_counter[(context, token)]
        count_of_context = float(len(self.context[context]))
        result = (count_of_token + 0.0001) / count_of_context
        return result
       

  def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
    # return token
            summ += map_to_probs[token]
            if summ > r:
                return token
  def generate_text(self, token_count: int):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        n = self.n
        context_queue = (n - 1) * ['<START>']
        result = []
        for _ in range(token_count):
            obj = self.random_token(tuple(context_queue))
            result.append(obj)
            if n > 1:
                context_queue.pop(0)
                if obj == '.':
                    context_queue = (n - 1) * ['<START>']
                else:
                    context_queue.append(obj)
        return ' '.join(result)

  def beamSearch_generator(self, token_count: int):
   n = self.n
   result, result1, result2, result3 = [], [], [],[]
   context_queue1 = (n - 1) * ['<START>']
   context_queue2 = (n - 1) * ['<START>']
   context_queue3 = (n - 1) * ['<START>']
   probability1, probability2, probability3, maximum = 0, 0 , 0, 0
   #first strin
   for _ in range(token_count):
      obj1 = self.random_token(tuple(context_queue1))
      result1.append(obj1)
      probability1 += self.prob(tuple(context_queue1),obj1)
      if n > 1:
          context_queue1.pop(0)
          if obj1 == '.':
              context_queue1 = (n - 1) * ['<START>']
          else:
                    context_queue1.append(obj1)
   # second string                 
   for _ in range(token_count):
      obj2 = self.random_token(tuple(context_queue2))
      result2.append(obj2)
      probability2 += self.prob(tuple(context_queue2),obj2)
      if n > 1:
          context_queue2.pop(0)
          if obj2 == '.':
              context_queue2 = (n - 1) * ['<START>']
          else:
              context_queue2.append(obj2)    
   # third string
   for _ in range(token_count):
      obj3 = self.random_token(tuple(context_queue3))
      result3.append(obj3)
      probability3 += self.prob(tuple(context_queue3),obj3)
      if n > 1:
          context_queue3.pop(0)
          if obj3 == '.':
              context_queue3 = (n - 1) * ['<START>']
          else:
              context_queue3.append(obj3)    

   maximum = max(probability1, probability2, probability3)
   if maximum == probability1:
     return ' '.join(result1)
   elif maximum == probability2:
     return ' '.join(result2)
   else:  
     return ' '.join(result3)


How can we add smoothing functionality?

Why didn't we return the most probable token? (stay tuned for the rest of the materials)






In [ ]:
def create_ngram_model(n, path):
    m = NgramModel(n)
    with open(path, 'r', encoding='utf-8') as f:
        text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
        text = text.split('.')
        text = random.sample(text, 20000)
        for sentence in text:
            # add back the fullstop
            sentence += '.'
            m.update(sentence)
    return m

In [ ]:
if __name__ == "__main__":
    start = time.time()
    m = create_ngram_model(6, './voa_fa_2003-2008_orig.txt')
    print (f'Language Model creating time: {time.time() - start}')
    start = time.time()
    #random.seed(44)
    print(f'{"="*50}\nGenerated text:')
    #print(m.generate_text(100))
    print(m.beamSearch_generator(100))
    print(f'{"="*50}')





Language Model creating time: 3.693009853363037
Generated text:
واشنگتن گفته است موضوع کمک های غذايی ارتباطی با گفت و گو ها در خصوص برنامه هسته ای پيونگ يانگ ندارد . با تشديد تظاهرات، خدمات مربوط به تلفن های موبايل نيزقطع شده است . نخست وزير اسرائيل همچنين قول داد از پارلمان درخواست کند اجازه دهد ۴۰۰ زندانی ديگر فلسطينی آزاد شوند . از سوی ديگر، روسيه می گويد که تامين سامانه هوايی اولويت همکاريهای نظامی با تهران است و همزمان کارشناسانی را برای راه اندازی سيستم دفاع هوايی تور - ام يک به ايران فرستاده است . اما، کن برمن اشاره می کند که آمريکا آمادگی آن را
